In [102]:
import pandas as pd
import numpy as np

In [103]:
links = pd.read_csv('ml-2016-small/links.csv', sep=',')
movies = pd.read_csv('ml-2016-small/movies.csv', sep=',', encoding='latin-1')
ratings = pd.read_csv('ml-2016-small/ratings.csv', sep=',')
tags = pd.read_csv('ml-2016-small/tags.csv', sep=',', encoding='latin-1')

# EDA

In [104]:
movies['genres'] = movies['genres'].str.replace('|', ',')
movies = movies.join(movies['genres'].str.get_dummies(','))

/tmp/ipykernel_280/3225289592.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  movies['genres'] = movies['genres'].str.replace('|', ',')


In [105]:
movies.head()

,movieId,title,genres,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),"Adventure,Animation,Children,Comedy,Fantasy",0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),"Adventure,Children,Fantasy",0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),"Comedy,Romance",0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),"Comedy,Drama,Romance",0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [106]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [107]:
ratings['rating'].describe()

count    100004.000000
mean          3.543608
std           1.058064
min           0.500000
25%           3.000000
50%           4.000000
75%           4.000000
max           5.000000
Name: rating, dtype: float64

In [108]:
movies['genres'].str.split(',').str.len().mean()

2.2290410958904108

In [109]:
movies['genres'].str.split(',').str.len().min()

1

In [110]:
movies['genres'].str.split(',').str.len().max()

10

# Part 1

## Code

In [111]:
ratings['norm_ratings'] = ratings['rating'] - 3.0

In [112]:
ratings.head()

,userId,movieId,rating,timestamp,norm_ratings
0,1,31,2.5,1260759144,-0.5
1,1,1029,3.0,1260759179,0.0
2,1,1061,3.0,1260759182,0.0
3,1,1129,2.0,1260759185,-1.0
4,1,1172,4.0,1260759205,1.0


In [113]:
#In norm_ratings, how many are positive,neutral and negative?
ratings['norm_ratings'].value_counts()

 1.0    28750
 0.0    20064
 2.0    15095
 0.5    10538
 1.5     7723
-1.0     7271
-0.5     4449
-2.0     3326
-1.5     1687
-2.5     1101
Name: norm_ratings, dtype: int64

In [114]:
ratings['genres'] = ratings['movieId'].map(movies.set_index('movieId')['genres'])
genre_ratings = ratings.drop(['movieId', 'rating', 'timestamp'], axis=1)
ratings = ratings.join(ratings['genres'].str.get_dummies(','))

In [115]:
ratings.head()

,userId,movieId,rating,timestamp,norm_ratings,genres,(no genres listed),Action,Adventure,Animation,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,31,2.5,1260759144,-0.5,Drama,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1029,3.0,1260759179,0.0,"Animation,Children,Drama,Musical",0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
2,1,1061,3.0,1260759182,0.0,Thriller,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1,1129,2.0,1260759185,-1.0,"Action,Adventure,Sci-Fi,Thriller",0,1,1,0,...,0,0,0,0,0,0,1,1,0,0
4,1,1172,4.0,1260759205,1.0,Drama,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [116]:
genre_ratings.head()

,userId,norm_ratings,genres
0,1,-0.5,Drama
1,1,0.0,"Animation,Children,Drama,Musical"
2,1,0.0,Thriller
3,1,-1.0,"Action,Adventure,Sci-Fi,Thriller"
4,1,1.0,Drama


In [117]:
genre_ratings_dict = {}
for index, row in genre_ratings.iterrows():
    for genre in row['genres'].split(','):
        if genre_ratings_dict.get((row['userId'], genre)) == None:
            genre_ratings_dict[(row['userId'], genre)] = row['norm_ratings']
        else:
            genre_ratings_dict[(row['userId'], genre)] += row['norm_ratings']

genre_ratings_dict

{(1, 'Drama'): -3.0,
 (1, 'Animation'): -3.0,
 (1, 'Children'): -1.0,
 (1, 'Musical'): 0.0,
 (1, 'Thriller'): -1.0,
 (1, 'Action'): -1.0,
 (1, 'Adventure'): -7.5,
 (1, 'Sci-Fi'): -3.0,
 (1, 'War'): -1.0,
 (1, 'Fantasy'): -3.5,
 (1, 'Horror'): 0.0,
 (1, 'Romance'): 0.5,
 (1, 'Comedy'): -5.0,
 (1, 'Crime'): -1.0,
 (1, 'Western'): 0.0,
 (2, 'Action'): 6.0,
 (2, 'Adventure'): 9.0,
 (2, 'Thriller'): 7.0,
 (2, 'Drama'): 24.0,
 (2, 'Romance'): 13.0,
 (2, 'Comedy'): 3.0,
 (2, 'Mystery'): 4.0,
 (2, 'Crime'): 6.0,
 (2, 'War'): 4.0,
 (2, 'IMAX'): 2.0,
 (2, 'Sci-Fi'): 4.0,
 (2, 'Horror'): 2.0,
 (2, 'Fantasy'): 4.0,
 (2, 'Western'): 4.0,
 (2, 'Children'): 4.0,
 (2, 'Animation'): 4.0,
 (2, 'Musical'): 3.0,
 (3, 'Adventure'): 3.0,
 (3, 'Children'): -1.5,
 (3, 'Fantasy'): 0.0,
 (3, 'Action'): 7.5,
 (3, 'Drama'): 23.0,
 (3, 'War'): 7.0,
 (3, 'Crime'): 7.0,
 (3, 'Comedy'): 11.0,
 (3, 'Thriller'): 6.0,
 (3, 'Romance'): 6.5,
 (3, 'Animation'): -0.5,
 (3, 'Musical'): -1.0,
 (3, 'Horror'): 1.5,
 (3, 'IMAX')

In [118]:
genre_ratings_df = pd.DataFrame.from_dict(genre_ratings_dict, orient='index')
genre_ratings_df = genre_ratings_df.reset_index().rename(columns={'index': 'userId', 0: 'norm_ratings'})
genre_ratings_df['userId'] = genre_ratings_df['userId'].astype('string')
genre_ratings_df['userId'] = genre_ratings_df['userId'].str.replace('(','')
genre_ratings_df['userId'] = genre_ratings_df['userId'].str.replace(')','')
genre_ratings_df['genre'] = genre_ratings_df['userId'].str.split(',').str[1]
genre_ratings_df['userId'] = genre_ratings_df['userId'].str.split(',').str[0]
genre_ratings_df = genre_ratings_df.join(genre_ratings_df['genre'].str.get_dummies(','))
for iter, rows in genre_ratings_df.iterrows():
    genre = genre_ratings_df['genre'].iloc[iter]
    genre_ratings_df[genre].iloc[iter] = genre_ratings_df[genre].iloc[iter] * genre_ratings_df['norm_ratings'].iloc[iter] 

genre_ratings_df = genre_ratings_df.drop(['genre', 'norm_ratings'], axis=1)
genre_ratings_df = genre_ratings_df.groupby(genre_ratings_df['userId']).sum()
genre_ratings_df = genre_ratings_df.set_index(genre_ratings_df.index.astype('int64'))
genre_ratings_df

/tmp/ipykernel_280/2874658781.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  genre_ratings_df['userId'] = genre_ratings_df['userId'].str.replace('(','')
/tmp/ipykernel_280/2874658781.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  genre_ratings_df['userId'] = genre_ratings_df['userId'].str.replace(')','')
/tmp/ipykernel_280/2874658781.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genre_ratings_df[genre].iloc[iter] = genre_ratings_df[genre].iloc[iter] * genre_ratings_df['norm_r

,'Action','Adventure','Animation','Children','Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','IMAX','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western','no genres listed'
userId,,,,,,,,,,,,,,,,,,,,
1,-1.0,-7.5,-3.0,-1.0,-5.0,-1.0,0.0,-3.0,-3.5,0.0,0.0,0.0,0.0,0.0,0.5,-3.0,-1.0,-1.0,0.0,0.0
10,14.0,12.0,0.0,0.0,7.0,4.0,0.0,16.0,3.0,0.0,3.0,0.0,0.0,3.0,6.0,7.0,8.0,1.0,-1.0,0.0
100,1.0,2.0,2.0,4.0,5.0,1.0,0.0,3.0,3.0,0.0,0.0,0.0,2.0,2.0,3.0,3.0,4.0,0.0,0.0,0.0
101,32.5,23.5,1.0,0.0,3.5,13.5,0.0,19.5,5.5,1.0,2.5,7.5,0.0,5.5,-0.5,26.5,16.0,4.5,-0.5,0.0
102,73.0,74.0,25.0,38.0,195.0,104.0,18.0,409.0,44.0,30.0,59.0,0.0,53.0,55.0,118.0,40.0,115.0,59.0,30.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,62.0,44.0,9.0,10.0,108.0,32.0,0.0,82.0,26.0,1.0,99.0,3.0,8.0,18.0,42.0,42.0,81.0,4.0,5.0,0.0
96,3.0,4.0,0.0,0.0,26.5,20.5,0.0,46.0,0.5,20.5,4.0,0.0,0.0,27.5,32.5,1.0,32.5,6.0,1.5,0.0
97,-15.5,3.5,-4.0,-5.5,17.0,4.0,0.0,1.5,0.0,1.0,-5.0,-6.0,-4.0,2.0,-3.5,0.5,0.0,-3.0,3.0,0.0


In [119]:
movies = movies.drop(['title', 'genres'], axis=1)
movies = movies.set_index(movies['movieId'])
movies = movies.drop(['movieId'], axis=1)

In [120]:
movies.insert(len(movies.columns)-1, 'no genres listed', movies.pop('(no genres listed)'))
movies

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,no genres listed
movieId,,,,,,,,,,,,,,,,,,,,
1,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
5,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162672,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
163056,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
163949,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


## Preducted score for each user for each movie

In [121]:
movies_array = movies.to_numpy()
user_recommendations = genre_ratings_df @ movies_array.T
user_recommendations

,0,1,2,3,4,5,6,7,8,9,...,9115,9116,9117,9118,9119,9120,9121,9122,9123,9124
userId,,,,,,,,,,,,,,,,,,,,,
1,-20.0,-12.0,-4.5,-7.5,-5.0,-3.0,-4.5,-8.5,-1.0,-9.5,...,-4.0,-11.5,-3.0,-3.0,-0.5,-10.0,-15.0,0.0,-5.0,0.0
10,22.0,15.0,13.0,29.0,7.0,26.0,13.0,12.0,14.0,34.0,...,27.0,36.0,16.0,16.0,14.0,34.0,36.0,0.0,7.0,0.0
100,16.0,9.0,8.0,11.0,5.0,6.0,8.0,6.0,1.0,7.0,...,7.0,6.0,3.0,3.0,7.0,8.0,9.0,0.0,5.0,0.0
101,33.5,29.0,3.0,22.5,3.5,62.0,3.0,23.5,32.5,72.0,...,38.0,85.0,19.5,19.5,15.5,42.5,88.0,0.0,3.5,0.0
102,376.0,156.0,313.0,722.0,195.0,292.0,313.0,112.0,73.0,262.0,...,583.0,246.0,409.0,409.0,233.0,601.0,231.0,18.0,195.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,197.0,80.0,150.0,232.0,108.0,175.0,150.0,54.0,62.0,187.0,...,262.0,247.0,82.0,82.0,123.0,168.0,174.0,0.0,108.0,0.0
96,31.0,4.5,59.0,105.0,26.5,56.0,59.0,4.0,3.0,39.5,...,82.5,12.0,46.0,46.0,65.0,82.5,8.5,0.0,26.5,0.0
97,11.0,-2.0,13.5,15.0,17.0,-11.5,13.5,-2.0,-15.5,-12.0,...,-3.5,-16.5,1.5,1.5,-3.5,1.5,-11.5,0.0,17.0,0.0


## Questions Part 1

In [122]:
user_top = user_recommendations.loc[user_recommendations.index == 526].to_numpy()[0].argsort()[-10:][::-1]
user = user_recommendations.loc[user_recommendations.index == 526].to_numpy()[0]
user_top

array([3891, 4872, 5845, 6378, 7648, 7575, 7004, 7951, 8737, 6890])

# Part 2

In [123]:
genre_count = movies.sum(axis=1)
genre_count

movieId
1         5
2         3
3         2
4         3
5         1
         ..
162672    3
163056    4
163949    1
164977    1
164979    1
Length: 9125, dtype: int64

In [124]:
movies_norm = movies.div(genre_count, axis=0)
movies_norm

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,no genres listed
movieId,,,,,,,,,,,,,,,,,,,,
1,0.00,0.200000,0.2,0.200000,0.200000,0.0,0.0,0.000000,0.200000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0
2,0.00,0.333333,0.0,0.333333,0.000000,0.0,0.0,0.000000,0.333333,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0
3,0.00,0.000000,0.0,0.000000,0.500000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.500000,0.00,0.0,0.0,0.0,0.0
4,0.00,0.000000,0.0,0.000000,0.333333,0.0,0.0,0.333333,0.000000,0.0,0.0,0.0,0.0,0.0,0.333333,0.00,0.0,0.0,0.0,0.0
5,0.00,0.000000,0.0,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162672,0.00,0.333333,0.0,0.000000,0.000000,0.0,0.0,0.333333,0.000000,0.0,0.0,0.0,0.0,0.0,0.333333,0.00,0.0,0.0,0.0,0.0
163056,0.25,0.250000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.250000,0.0,0.0,0.0,0.0,0.0,0.000000,0.25,0.0,0.0,0.0,0.0
163949,0.00,0.000000,0.0,0.000000,0.000000,0.0,1.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0


## Normalized features

In [125]:
movies_norm_array = movies_norm.to_numpy()
user_recommendations_norm = genre_ratings_df @ movies_norm_array.T
user_recommendations_norm

,0,1,2,3,4,5,6,7,8,9,...,9115,9116,9117,9118,9119,9120,9121,9122,9123,9124
userId,,,,,,,,,,,,,,,,,,,,,
1,-4.0,-4.000000,-2.25,-2.500000,-5.0,-1.000000,-2.25,-4.25,-1.0,-3.166667,...,-1.333333,-2.875,-3.0,-3.0,-0.25,-3.333333,-3.750,0.0,-5.0,0.0
10,4.4,5.000000,6.50,9.666667,7.0,8.666667,6.50,6.00,14.0,11.333333,...,9.000000,9.000,16.0,16.0,7.00,11.333333,9.000,0.0,7.0,0.0
100,3.2,3.000000,4.00,3.666667,5.0,2.000000,4.00,3.00,1.0,2.333333,...,2.333333,1.500,3.0,3.0,3.50,2.666667,2.250,0.0,5.0,0.0
101,6.7,9.666667,1.50,7.500000,3.5,20.666667,1.50,11.75,32.5,24.000000,...,12.666667,21.250,19.5,19.5,7.75,14.166667,22.000,0.0,3.5,0.0
102,75.2,52.000000,156.50,240.666667,195.0,97.333333,156.50,56.00,73.0,87.333333,...,194.333333,61.500,409.0,409.0,116.50,200.333333,57.750,18.0,195.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,39.4,26.666667,75.00,77.333333,108.0,58.333333,75.00,27.00,62.0,62.333333,...,87.333333,61.750,82.0,82.0,61.50,56.000000,43.500,0.0,108.0,0.0
96,6.2,1.500000,29.50,35.000000,26.5,18.666667,29.50,2.00,3.0,13.166667,...,27.500000,3.000,46.0,46.0,32.50,27.500000,2.125,0.0,26.5,0.0
97,2.2,-0.666667,6.75,5.000000,17.0,-3.833333,6.75,-1.00,-15.5,-4.000000,...,-1.166667,-4.125,1.5,1.5,-1.75,0.500000,-2.875,0.0,17.0,0.0


## Questions Part 2

In [126]:
user2_top = user_recommendations_norm.loc[user_recommendations_norm.index == 526].to_numpy()[0].argsort()[-10:][::-1]
user2 = user_recommendations_norm.loc[user_recommendations_norm.index == 526].to_numpy()[0]
user2[user2_top]

array([14.5, 14.5, 14.5, 14.5, 14.5, 14.5, 14.5, 14.5, 14.5, 14.5])

In [127]:
user3_top = user_recommendations_norm.loc[user_recommendations_norm.index == 14].to_numpy()[0].argsort()[-10:][::-1]
user3 = user_recommendations_norm.loc[user_recommendations_norm.index == 14].to_numpy()[0]
user3[user3_top]

array([3., 3., 3., 3., 3., 3., 3., 3., 3., 3.])

# Part 3

In [128]:
genre_doc_frec = {}
for index, row in genre_ratings.iterrows():
    for genre in row['genres'].split(','):
        if genre_doc_frec.get(genre) == None:
            genre_doc_frec[genre] = 1
        else:
            genre_doc_frec[genre] += 1

genre_inv_doc_frec = {k: 1/v for k, v in genre_doc_frec.items()}
genre_inv_doc_frec

{'Drama': 2.234537003932785e-05,
 'Animation': 0.00016207455429497568,
 'Children': 0.0001152073732718894,
 'Musical': 0.00021177467174925878,
 'Thriller': 3.961965134706815e-05,
 'Action': 3.696037847427558e-05,
 'Adventure': 4.54194486078939e-05,
 'Sci-Fi': 6.508298080052066e-05,
 'War': 0.0001990049751243781,
 'Fantasy': 9.38350380031904e-05,
 'Horror': 0.00014727540500736376,
 'Romance': 5.17170045510964e-05,
 'Comedy': 2.6297796244674696e-05,
 'Crime': 6.147792942333702e-05,
 'Western': 0.0005230125523012552,
 'Mystery': 0.00013114754098360657,
 'IMAX': 0.0003168567807351077,
 'Documentary': 0.0006393861892583121,
 'Film-Noir': 0.0008771929824561404,
 '(no genres listed)': 0.05555555555555555}

In [129]:
genre_doc_frec_sorted = sorted(genre_doc_frec.items(), key=lambda x: x[1], reverse=True)
print(genre_doc_frec_sorted[:5])
print(genre_doc_frec_sorted[-5:])

[('Drama', 44752), ('Comedy', 38026), ('Action', 27056), ('Thriller', 25240), ('Adventure', 22017)]
[('IMAX', 3156), ('Western', 1912), ('Documentary', 1564), ('Film-Noir', 1140), ('(no genres listed)', 18)]


In [130]:
idf_df = pd.DataFrame(0, columns=genre_inv_doc_frec.keys(), index=[0])
idf_df.iloc[0] = genre_inv_doc_frec.values()
idf_df

,Drama,Animation,Children,Musical,Thriller,Action,Adventure,Sci-Fi,War,Fantasy,Horror,Romance,Comedy,Crime,Western,Mystery,IMAX,Documentary,Film-Noir,(no genres listed)
0,0.000022,0.000162,0.000115,0.000212,0.00004,0.000037,0.000045,0.000065,0.000199,0.000094,0.000147,0.000052,0.000026,0.000061,0.000523,0.000131,0.000317,0.000639,0.000877,0.055556


In [131]:
genre_ratings_df

,'Action','Adventure','Animation','Children','Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','IMAX','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western','no genres listed'
userId,,,,,,,,,,,,,,,,,,,,
1,-1.0,-7.5,-3.0,-1.0,-5.0,-1.0,0.0,-3.0,-3.5,0.0,0.0,0.0,0.0,0.0,0.5,-3.0,-1.0,-1.0,0.0,0.0
10,14.0,12.0,0.0,0.0,7.0,4.0,0.0,16.0,3.0,0.0,3.0,0.0,0.0,3.0,6.0,7.0,8.0,1.0,-1.0,0.0
100,1.0,2.0,2.0,4.0,5.0,1.0,0.0,3.0,3.0,0.0,0.0,0.0,2.0,2.0,3.0,3.0,4.0,0.0,0.0,0.0
101,32.5,23.5,1.0,0.0,3.5,13.5,0.0,19.5,5.5,1.0,2.5,7.5,0.0,5.5,-0.5,26.5,16.0,4.5,-0.5,0.0
102,73.0,74.0,25.0,38.0,195.0,104.0,18.0,409.0,44.0,30.0,59.0,0.0,53.0,55.0,118.0,40.0,115.0,59.0,30.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,62.0,44.0,9.0,10.0,108.0,32.0,0.0,82.0,26.0,1.0,99.0,3.0,8.0,18.0,42.0,42.0,81.0,4.0,5.0,0.0
96,3.0,4.0,0.0,0.0,26.5,20.5,0.0,46.0,0.5,20.5,4.0,0.0,0.0,27.5,32.5,1.0,32.5,6.0,1.5,0.0
97,-15.5,3.5,-4.0,-5.5,17.0,4.0,0.0,1.5,0.0,1.0,-5.0,-6.0,-4.0,2.0,-3.5,0.5,0.0,-3.0,3.0,0.0


In [132]:
user_ratings_idf = pd.DataFrame(genre_ratings_df.values * idf_df.values, columns=genre_ratings_df.columns, index=genre_ratings_df.index)
user_ratings_idf

,'Action','Adventure','Animation','Children','Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','IMAX','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western','no genres listed'
userId,,,,,,,,,,,,,,,,,,,,
1,-0.000022,-0.001216,-0.000346,-0.000212,-0.000198,-0.000037,0.000000,-0.000195,-0.000697,0.000000,0.000000,0.000000,0.000000,0.000000,0.000262,-0.000393,-0.000317,-0.000639,0.000000,0.0
10,0.000313,0.001945,0.000000,0.000000,0.000277,0.000148,0.000000,0.001041,0.000597,0.000000,0.000442,0.000000,0.000000,0.000184,0.003138,0.000918,0.002535,0.000639,-0.000877,0.0
100,0.000022,0.000324,0.000230,0.000847,0.000198,0.000037,0.000000,0.000195,0.000597,0.000000,0.000000,0.000000,0.000053,0.000123,0.001569,0.000393,0.001267,0.000000,0.000000,0.0
101,0.000726,0.003809,0.000115,0.000000,0.000139,0.000499,0.000000,0.001269,0.001095,0.000094,0.000368,0.000388,0.000000,0.000338,-0.000262,0.003475,0.005070,0.002877,-0.000439,0.0
102,0.001631,0.011994,0.002880,0.008047,0.007726,0.003844,0.000818,0.026619,0.008756,0.002815,0.008689,0.000000,0.001394,0.003381,0.061715,0.005246,0.036439,0.037724,0.026316,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.001385,0.007131,0.001037,0.002118,0.004279,0.001183,0.000000,0.005337,0.005174,0.000094,0.014580,0.000155,0.000210,0.001107,0.021967,0.005508,0.025665,0.002558,0.004386,0.0
96,0.000067,0.000648,0.000000,0.000000,0.001050,0.000758,0.000000,0.002994,0.000100,0.001924,0.000589,0.000000,0.000000,0.001691,0.016998,0.000131,0.010298,0.003836,0.001316,0.0
97,-0.000346,0.000567,-0.000461,-0.001165,0.000674,0.000148,0.000000,0.000098,0.000000,0.000094,-0.000736,-0.000310,-0.000105,0.000123,-0.001831,0.000066,0.000000,-0.001918,0.002632,0.0


## Accounting for differences in frequecies 

In [133]:
user_recommendations_idf = user_ratings_idf @ movies_norm_array.T
user_recommendations_idf

,0,1,2,3,4,5,6,7,8,9,...,9115,9116,9117,9118,9119,9120,9121,9122,9123,9124
userId,,,,,,,,,,,,,,,,,,,,,
1,-0.000534,-0.000708,0.000032,-0.000044,-0.000198,-0.000125,0.000032,-0.000714,-0.000022,-0.000518,...,-0.000171,-0.000408,-0.000195,-0.000195,-0.000028,-0.000383,-0.000582,0.000000,-0.000198,0.000000
10,0.000564,0.000847,0.001708,0.001486,0.000277,0.000999,0.001708,0.000972,0.000313,0.001598,...,0.001339,0.000904,0.001041,0.001041,0.002836,0.002041,0.000943,0.000000,0.000277,0.000000
100,0.000439,0.000589,0.000884,0.000654,0.000198,0.000442,0.000884,0.000586,0.000022,0.000538,...,0.000488,0.000185,0.000195,0.000195,0.001418,0.000696,0.000334,0.000000,0.000198,0.000000
101,0.001031,0.001634,-0.000061,0.000382,0.000139,0.002098,-0.000061,0.001904,0.000726,0.003202,...,0.002236,0.002095,0.001269,0.001269,0.002404,0.001605,0.002276,0.000000,0.000139,0.000000
102,0.007881,0.009599,0.034721,0.032020,0.007726,0.013971,0.034721,0.010020,0.001631,0.016688,...,0.023916,0.006890,0.026619,0.026619,0.049077,0.033443,0.006907,0.000818,0.007726,0.000818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.003948,0.004808,0.013123,0.010527,0.004279,0.009411,0.013123,0.004625,0.001385,0.011394,...,0.015194,0.007151,0.005337,0.005337,0.023816,0.011478,0.004800,0.000000,0.004279,0.000000
96,0.000360,0.000249,0.009024,0.007014,0.001050,0.003708,0.009024,0.000324,0.000067,0.003671,...,0.004627,0.000359,0.002994,0.002994,0.013648,0.006880,0.000236,0.000000,0.001050,0.000000
97,-0.000077,-0.000199,-0.000579,-0.000353,0.000674,-0.000066,-0.000579,-0.000299,-0.000346,0.000074,...,-0.000213,-0.000112,0.000098,0.000098,-0.000915,-0.000389,0.000072,0.000000,0.000674,0.000000


## Questions Part 3

In [134]:
user4_top = user_recommendations_idf.loc[user_recommendations_idf.index == 526].to_numpy()[0].argsort()[-10:][::-1]
user4 = user_recommendations_idf.loc[user_recommendations_idf.index == 526].to_numpy()[0]
user4[user4_top]

array([0.00459442, 0.00459442, 0.00459442, 0.00459442, 0.00459442,
       0.00459442, 0.00459442, 0.00459442, 0.00459442, 0.00459442])

# Part 4

In [179]:
from sklearn.metrics.pairwise import cosine_similarity

def mmr(relatedItems, query, numRecommendations):
    alpha = 0.5
    diversifiedItems = np.zeros(20)
    count = 0
    while count < numRecommendations:
        bestItem = []
        for item in relatedItems:
            relevanceToQuery = cosine_similarity(movies.iloc[item].to_numpy().reshape(1, -1), query)
            similarityWithSelectedItem = cosine_similarity(movies.iloc[item].to_numpy().reshape(1, -1), diversifiedItems.reshape(1, -1))
            combinedScore = alpha * relevanceToQuery - (1 - alpha)*similarityWithSelectedItem
            
            if bestItem == []:
                bestItem = [combinedScore, item]
            elif combinedScore > bestItem[0]:
                bestItem = [combinedScore, item]
        
        print(bestItem)
        diversifiedItems[count] = bestItem[1]
        relatedItems = np.delete(relatedItems, np.where(relatedItems == bestItem[1]))
        count += 1
    
    return diversifiedItems


In [136]:
movies.iloc[484].to_numpy().reshape(1, -1).shape

(1, 20)

In [137]:
idf_df.to_numpy().shape

(1, 20)

In [173]:
user5_top = user_recommendations_idf.loc[user_recommendations_idf.index == 526].to_numpy()[0].argsort()[-200:][::-1]

In [180]:
mmr(user5_top, idf_df.to_numpy(), 10)

[array([[0.00534339]]), 3418]
[array([[0.00534339]]), 9119]
[array([[0.00534339]]), 1319]
[array([[0.00534339]]), 4492]
[array([[0.00534339]]), 4068]
[array([[0.00504413]]), 8254]
[array([[0.00504413]]), 7751]
[array([[0.00285091]]), 484]
[array([[0.00285091]]), 9083]
[array([[0.00285091]]), 1402]


array([3418., 9119., 1319., 4492., 4068., 8254., 7751.,  484., 9083.,
       1402.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.])

In [181]:
user6_top = user_recommendations_idf.loc[user_recommendations_idf.index == 225].to_numpy()[0].argsort()[-200:][::-1]

In [182]:
mmr(user6_top, idf_df.to_numpy(), 10)

[array([[0.00890834]]), 2692]
[array([[0.00890834]]), 3234]
[array([[0.00890834]]), 782]
[array([[0.00890834]]), 4330]
[array([[0.00790284]]), 6488]
[array([[0.00739538]]), 9057]
[array([[0.00739538]]), 1136]
[array([[0.00739538]]), 791]
[array([[0.00575285]]), 1174]
[array([[0.00575285]]), 4647]


array([2692., 3234.,  782., 4330., 6488., 9057., 1136.,  791., 1174.,
       4647.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.])

In [205]:
user7_top = user_recommendations_idf.loc[user_recommendations_idf.index == 341].to_numpy()[0].argsort()[-200:][::-1]
mmr(user7_top, idf_df.to_numpy(), 10)

[array([[0.00739538]]), 1136]
[array([[0.00739538]]), 791]
[array([[0.00739538]]), 9057]
[array([[0.00637638]]), 2341]
[array([[0.00637638]]), 2187]
[array([[0.00637638]]), 1506]
[array([[0.00637638]]), 4317]
[array([[0.00637638]]), 4329]
[array([[0.00575285]]), 2137]
[array([[0.00575285]]), 2136]


array([1136.,  791., 9057., 2341., 2187., 1506., 4317., 4329., 2137.,
       2136.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.])

In [204]:
for l in range(10, 200, 10):
    print(str(l) + ":")
    user7_top = user_recommendations_idf.loc[user_recommendations_idf.index == 341].to_numpy()[0].argsort()[-l:][::-1]
    mmr(user7_top, idf_df.to_numpy(), 10)

10:
[array([[0.00470579]]), 2166]
[array([[0.00470579]]), 6629]
[array([[0.00470579]]), 5108]
[array([[0.00470579]]), 1204]
[array([[0.00470579]]), 913]
[array([[0.00470579]]), 5597]
[array([[0.00470579]]), 3552]
[array([[0.00470579]]), 557]
[array([[0.00470579]]), 1192]
[array([[0.00470579]]), 3225]
20:
[array([[0.00739538]]), 1136]
[array([[0.00739538]]), 791]
[array([[0.00739538]]), 9057]
[array([[0.00575285]]), 2137]
[array([[0.00575285]]), 2136]
[array([[0.00575285]]), 2671]
[array([[0.00470579]]), 2166]
[array([[0.00470579]]), 6629]
[array([[0.00470579]]), 5108]
[array([[0.00470579]]), 1204]
30:
[array([[0.00739538]]), 1136]
[array([[0.00739538]]), 791]
[array([[0.00739538]]), 9057]
[array([[0.00637638]]), 2341]
[array([[0.00637638]]), 2187]
[array([[0.00637638]]), 1506]
[array([[0.00637638]]), 4317]
[array([[0.00637638]]), 4329]
[array([[0.00575285]]), 2137]
[array([[0.00575285]]), 2136]
40:
[array([[0.00739538]]), 1136]
[array([[0.00739538]]), 791]
[array([[0.00739538]]), 9057]